In [2]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plotly_kp_pc_inliers(img_l, kp_l, kp_r, pc, inliers_bool, title=""):
    fig = make_subplots(rows=2, cols=1,
                        specs=[
                            [{"type":"xy"}],
                            [{"type":"scene"}] 
                        ],
                        vertical_spacing=0.05,
                        row_heights=[0.4,0.6]
                       )
    fig.update_layout(margin=dict(l=20, r=250, b=0, t=40), autosize=False, width=1100, height=1200)
    
    # IMAGES TRACES fig.data[0]
    img_l_trace  = go.Heatmap(z=img_l, colorscale='gray',name="img_l")

    fig.add_trace(img_l_trace, row=1, col=1)

    fig.update_yaxes(autorange='reversed', scaleanchor='x', constrain='domain', showgrid=False, zeroline=False, row=1, col=1)
    fig.update_xaxes(constrain='domain',showgrid=False, zeroline=False, row=1, col=1)

    fig.data[0].showscale = False; fig.data[0].coloraxis = None


    # KEYPOINTS TRACES
    small_size=5
    num_inliers = sum(inliers_bool); num_outliers = len(inliers_bool) - num_inliers
    inliers_sizes = np.full(num_inliers, small_size); outliers_sizes = np.full(num_outliers, small_size)
    kp_l_inliers = kp_l[:, inliers_bool]; kp_l_inliers_text = [f'({x:.1f}, {y:.1f})' for x, y in kp_l_inliers.T]

    kp_l_outliers = kp_l[:, ~inliers_bool]; kp_l_outliers_text = [f'({x:.1f}, {y:.1f})' for x, y in kp_l_outliers.T]

    kp_r_inliers = kp_r[:, inliers_bool]; kp_r_inliers_text = [f'({x:.1f}, {y:.1f})' for x, y in kp_r_inliers.T]

    kp_r_outliers = kp_r[:, ~inliers_bool]; kp_r_outliers_text = [f'({x:.1f}, {y:.1f})' for x, y in kp_r_outliers.T]

    inliers_pc = pc[:, inliers_bool]; inliers_pc_txt = [f'({x:.1f}, {y:.1f}, {z:.1f})' for x, y, z in inliers_pc.T]

    outliers_pc = pc[:, ~inliers_bool]; outliers_pc_txt = [f'({x:.1f}, {y:.1f}, {z:.1f})' for x, y, z in outliers_pc.T]

    kp_l_inliers_tt = [f'pc={x}<br>kp_r={y}' for x,y in zip(inliers_pc_txt, kp_r_inliers_text)]
    kp_l_inliers_scat = go.Scatter(x=kp_l_inliers[0], y=kp_l_inliers[1], mode="markers", name="kp_l_inliers",
                                   marker=dict(color="green", size=inliers_sizes),
                                   hovertemplate="(%{x:.1f}, %{y:.1f}), %{text}", text=kp_l_inliers_tt)
    kp_l_outliers_tt = [f'pc={x}<br>kp_r={y}' for x, y in zip(outliers_pc_txt, kp_r_outliers_text)]
    kp_l_outliers_scat = go.Scatter(x=kp_l_outliers[0], y=kp_l_outliers[1], mode="markers", name="kp_l_outliers",
                                    marker=dict(color="red", size=outliers_sizes),
                                    hovertemplate="(%{x:.1f}, %{y:.1f}), %{text}",text=kp_l_outliers_tt)
    # ADD TRACES kp_l_inliers 1, kp_l_outliers 2
    fig.add_traces([kp_l_inliers_scat, kp_l_outliers_scat], rows=[1, 1], cols=[1, 1])

    # PC TRACE
    inliers_pc_tt_txt = [f'kp_l={l}, kp_r={r}' for l,r in zip(kp_l_inliers_text, kp_r_inliers_text)]
    inliers_pc_trace = go.Scatter3d(x=inliers_pc[0], y=inliers_pc[2], z=inliers_pc[1], mode='markers',
                                    name="inliers_pc",
                                    marker=dict(color="green", size=inliers_sizes),
                                    hovertemplate="(%{x:.1f}, %{z:.1f}, %{y:.1f})" + "<br>%{text}",
                                    text=inliers_pc_tt_txt)

    outliers_pc_tt_txt = [f'kp_l={l}, kp_r={r}' for l, r in zip(kp_l_outliers_text, kp_r_outliers_text)]
    outliers_pc_trace = go.Scatter3d(x=outliers_pc[0], y=outliers_pc[2], z=outliers_pc[1], mode='markers',
                                     name="outliers_pc",
                                     marker=dict(color="red", size=outliers_sizes),
                                     hovertemplate="(%{x:.1f}, %{z:.1f}, %{y:.1f})" + "<br>%{text}",
                                     text=outliers_pc_tt_txt)

    # ADD TRACES kp_r_inliers 3, kp_r_outliers_scat 4
    fig.add_traces([inliers_pc_trace, outliers_pc_trace], rows=[2, 2], cols=[1, 1])

    fig.update_scenes(zaxis_autorange="reversed")
    camera = dict(up=dict(x=0, y=0, z=1),
                  center=dict(x=0, y=0, z=0),
                  eye=dict(x=0.2, y=-2, z=0.5))
    fig.update_layout(scene_camera=camera)
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Z', zaxis_title='Y'))

    # GENERAL
    fig.update_layout(coloraxis_showscale=False)
    fig.update_layout(showlegend=True)
    fig.update_layout(title_text=title, title_x=0.5)

    fw = go.FigureWidget(fig)

    inliers_traces, outliers_traces = [],[]
    for trace in fw.data:
        if 'inliers' in trace.name: 
            inliers_traces.append(trace)
        elif 'outliers' in trace.name:
            outliers_traces.append(trace)
    
    # ON_HOVER
    def update_point(trace, points, state):
        if not points.point_inds:
            return
        i = points.point_inds[0]
        with fw.batch_update():
            if trace.name in ['inliers_pc', 'kp_l_inliers']:
                for trace in inliers_traces:
                    new_sizes = np.copy(inliers_sizes)
                    new_sizes[i] = 15
                    trace.marker.size = new_sizes
                for trace in outliers_traces:
                    new_sizes = np.copy(outliers_sizes); trace.marker.size = new_sizes

            elif trace.name in ['outliers_pc', 'kp_l_outliers']:
                for trace in outliers_traces:
                    new_sizes = np.copy(outliers_sizes)
                    new_sizes[i] = 15
                    trace.marker.size = new_sizes
                for trace in inliers_traces:
                    new_sizes = np.copy(inliers_sizes); trace.marker.size = new_sizes

    for trace in (inliers_traces + outliers_traces):
        trace.on_hover(update_point)
    
    return fw

import kitti, triang,features
featurez = features.Features()
n = 2201
q=0.98; cont=0.02
k, ext_id, ext_l_to_r = kitti.read_cameras()  # k=(3,4) ext_l0/r0 (4,4)
kp_ln, desc_ln, kp_rn = featurez.get_kps_desc_stereo_pair(n)
kp_ln, kp_rn, pc_lr_n_in_ln, desc_ln = triang.triang_and_rel_filter(kp_ln, kp_rn, k, ext_id, ext_l_to_r, desc_ln)
comb_n_filter = triang.quant_and_forest(pc_lr_n_in_ln, q=q, cont=cont)
img_ln, img_rn = kitti.read_images(n)
fw = plotly_kp_pc_inliers(img_ln, kp_ln, kp_rn, pc_lr_n_in_ln, comb_n_filter, title=f"f={n}, comb, in l{n}, q={q}, cont={cont}")
fw

FigureWidget({
    'data': [{'colorscale': [[0.0, 'rgb(0, 0, 0)'], [0.09090909090909091, 'rgb(16,
            …